In [85]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# LCMS Model Training Data Setup

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/redcastle-resources/lcms-training/blob/main/4-Training_Data_Setup.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/redcastle-resources/lcms-training/blob/main/4-Training_Data_Setup.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://github.com/redcastle-resources/lcms-training/blob/main/4-Training_Data_Setup.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>
</table>
<br/><br/><br/>

## Overview


This notebook takes reference data from the TimeSync image interpretation tool and extracts the values from the LandTrendr and CCDC outputs from the previous notebooks as well as elevation data to create the training data tables needed for LCMS model calibration


### Objective

In this tutorial, you learn how to create model training featureCollections in GEE using methods that are unlikely to run out of memory

This tutorial uses the following Google Cloud services:

- `Google Earth Engine`

The steps performed include:

- Bringing in TimeSync data
- Creating terrain predictors
- Extracting LandTrendr, CCDC, and terrain values for TimeSync plot locations as assets

In [1]:
#Module imports
#!python -m pip install geeViz --upgrade
try:
    import geeViz.getImagesLib as getImagesLib
except:
    !python -m pip install geeViz
    import geeViz.getImagesLib as getImagesLib

import geeViz.changeDetectionLib as changeDetectionLib
import geeViz.assetManagerLib as aml
import geeViz.taskManagerLib as tml
import geeViz.gee2Pandas as g2p

# from IPython.display import IFrame,display, HTML
ee = getImagesLib.ee
Map = getImagesLib.Map

# Can set the port used for viewing map outputs
Map.port = 1234
print('Done')


  Obtaining dependency information for geeViz from https://files.pythonhosted.org/packages/3e/6b/b7eb40bfc276c6dbe7718fead624c52c1f0b4b0617e141f02236b21df278/geeViz-2023.8.4-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.0/246.0 kB 5.7 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.3/102.3 kB 17.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Obtaining dependency information for google-api-python-client>=1.12.1 from https://files.pythonhosted.org/packages/49/b1/a1364b88787702a308db814742b81fd3b57e4a616c0bbb50a9a3ad97184b/google_api_python_client-2.97.0-py2.py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 681.6/681.6 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 79.0 MB/s eta 0:00:00:00:0100:01
  Created wheel for earthengine-api: filename=earthengine_api-0.1.363-py3-none-any.whl size=275891 sha256=58c6e4a83c9f817f72fea374

Enter verification code:  4/1Adeu5BVmnvzH2wFrmkZ6qVmV137dkQygAmGbePsqedvXSICFThSadA0NsTw



Successfully saved authorization token.
Successfully initialized
geeViz package folder: /opt/conda/lib/python3.10/site-packages/geeViz
PyTables is not installed. No support for HDF output.
Done


## Before you begin

### Set your current URL under `workbench_url`
* This will be in your URL/search bar at the top of the browser window you are currently in
* It will look something like `https://1234567890122-dot-us-west3.notebooks.googleusercontent.com/`

### Set a folder to use for all exports under `export_path_root` 
* It will be something like `projects/projectID/assets/someFolder`
* This folder does not have to already exist. If it does not exist, it will be created

In [6]:
workbench_url = 'https://53c21733d8125e22-dot-us-west3.notebooks.googleusercontent.com/'
export_path_root  = 'projects/rcr-gee/assets/lcms-training'

print('Done')

Done


In [7]:
# Bring in all folders/collections that are needed
# These must already exist as they are created in previous notebooks
export_composite_collection = f'{export_path_root}/lcms-training_module-2_composites'
export_landTrendr_collection = f'{export_path_root}/lcms-training_module-3_landTrendr'
export_ccdc_collection = f'{export_path_root}/lcms-training_module-3_CCDC'

export_timeSync_folder = f'{export_path_root}/lcms-training_module-4_timeSync'

# This is the pre-made TimeSync data
# Creating this dataset is not covered in this set of notebooks
timeSync_featureCollection = 'projects/lcms-292214/assets/R8/PR_USVI/TimeSync/18_PRVI_AllPlots_TimeSync_Annualized_Table_secLC'


aml.create_asset(export_timeSync_folder)

# Currently geeView within Colab uses a different project to authenticate through, so you may need to make your asset public to view from within Colab
aml.updateACL(export_timeSync_folder,writers = [],all_users_can_read = True,readers = [])

print('Done')

Found the following sub directories:  ['lcms-training', 'lcms-training_module-4_timeSync']
Will attempt to create them if they do not exist
Asset projects/rcr-gee/assets/lcms-training already exists
Asset projects/rcr-gee/assets/lcms-training/lcms-training_module-4_timeSync already exists
Updating permissions for:  projects/rcr-gee/assets/lcms-training/lcms-training_module-4_timeSync
Done


In [11]:
Map.proxy_url = workbench_url
# First, let's bring in TimeSync and set up some parameters
# Set the projection
# Even though we're working outside the Conterminous United States, Puerto Rico and the US Virgin Islands
# uses the same USGS Albers projection
crs = getImagesLib.common_projections['NLCD_CONUS']['crs']
transform  = getImagesLib.common_projections['NLCD_CONUS']['transform']
scale = None

# Bring in annualized TimeSync data
# Each plot has one feature for each year from 2000 to 2020
# This results in over 20k plots
timeSyncData = ee.FeatureCollection(timeSync_featureCollection)
training_years = [int(yr) for yr in timeSyncData.aggregate_histogram('YEAR').keys().getInfo()]
plot_ids = timeSyncData.aggregate_histogram('PLOTID').keys().getInfo()
print('Training years:',training_years)
print('Unique Plot ID Count:',len(plot_ids))
print('Total Training Plots:', timeSyncData.size().getInfo())
print('TimeSync Attributes:',timeSyncData.first().propertyNames().getInfo())

print('Done')

Training years: [2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020]
Unique Plot ID Count: 1000
Total Training Plots: 20992
TimeSync Attributes: ['DOM_LU', 'GRASS-TS', 'latitude', 'JULIANDAY', 'COMMENTS', 'BARREN-IMP', 'SEC_LC', 'SHRUBS-TRE', 'INTERP', 'YEAR', 'SHRUBS', 'X', 'GRASS', 'Y', 'SEC_LU', 'SEC_LU_EL', 'DOM_SEC_LC', 'BARREN-GRA', 'DOM_LU_EL', 'IMPERVIOUS', 'WATER', 'longitude', 'CP_NOTES', 'SNOW', 'BARREN-TS', 'GRASS-TREE', 'PLOTID', 'CP', 'LAST_MODIF', 'BARREN-TRE', 'GRASS-SHRU', 'PACKET_ID', 'BARREN', 'DOM_LC', 'TREES', 'BARREN-SHR', 'CHANGE_DUR', 'SHRUBS-TS', 'system:index', 'TS-TREES', 'TS']
Done


In [9]:
Map.clearMap()

# Let's view the TimeSync data
# This will zoom to the first plot in the featureCollection
# Find a plot (they're 30m x 30m - small), and double-click on it to see 
# that there are actually multiple plots at that location - 1 for each year
Map.addLayer(timeSyncData.map(lambda f:f.buffer(15).bounds(5,crs)),{'layerType':'geeVectorImage'},'LCMS PRUSVI TimeSync Training Data')
Map.turnOnInspector()
Map.centerObject(timeSyncData.first())
Map.view()

Adding layer: LCMS PRUSVI TimeSync Training Data
Starting webmap
Using default refresh token for geeView: /home/jupyter/.config/earthengine/credentials
Local web server at: http://localhost:1234/geeView/ already serving.
cwd /home/jupyter/lcms-training
Workbench Proxy URL: https://53c21733d8125e22-dot-us-west3.notebooks.googleusercontent.com/proxy/1234/geeView/?accessToken=None


127.0.0.1 - - [16/Aug/2023 17:36:32] "GET /geeView/?accessToken=None HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2023 17:36:32] "GET /geeView/css/style.min.css HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2023 17:36:32] "GET /geeView/js/gena-gee-palettes.js HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2023 17:36:32] "GET /geeView/js/load.min.js HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2023 17:36:32] "GET /geeView/js/lcms-viewer.min.js HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2023 17:36:32] "GET /geeView/js/runGeeViz.js HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2023 17:36:33] "GET /geeView/images/GEE_logo_transparent.png HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2023 17:36:33] "GET /geeView/images/menu-hamburger_ffffff.svg HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2023 17:36:33] "GET /geeView/images/usdalogo.png HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2023 17:36:33] "GET /geeView/images/layer_icon.png HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2023 17:36:33] "GET /geeView/images/GEE.png HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2023 1

* The challenge is we have to pull all relevant predictor variables for each year from LandTrendr and CCDC without running out of memory
* If we pulled the predictors, trained a model, and applied it all at once, chances are very high we would run out of memory and it would fail
* In order to reduce the possibility of memory errors, we will extract predictors from our TimeSync plot locations for each year and export them individually as a featureCollection asset


In [10]:
 # The TimeSync data has the reference classes, but lacks the predictor variables
# We will now set up annual LandTrendr and CCDC outputs from our raw exported assets


# First, we will bring in LandTrendr data for each year

landTrendrCollection = ee.ImageCollection(export_landTrendr_collection)

landTrendrCollectionInfo = landTrendrCollection.first().toDictionary().getInfo()

Map.clearMap()

# View exported LT output
# Convert stacked outputs into collection of fitted, magnitude, slope, duration, etc values for each year
# While the fitted LandTrendr value is generally of most importance to our models, 
# LandTrendr segment duration, slope, and magnitude of change can also help our models
lt_fit = changeDetectionLib.batchSimpleLTFit(landTrendrCollection,landTrendrCollectionInfo['startYear'],landTrendrCollectionInfo['endYear'],None,bandPropertyName='band',arrayMode=True)

# Vizualize image collection for charting (opacity set to 0 so it will chart but not be visible)
Map.addLayer(lt_fit,{'bands':'swir2_LT_fitted,nir_LT_fitted,red_LT_fitted','min':0.15,'max':0.6},'LandTrendr All Predictors Time Series')

# Visualize fitted landTrendr composites
fitted_bns = lt_fit.select(['.*_fitted']).first().bandNames()
out_bns = fitted_bns.map(lambda bn: ee.String(bn).split('_').get(0))

# Give same names as composites
lt_synth = lt_fit.select(fitted_bns,out_bns)

# Visualize LandTrendr fitted 
Map.addTimeLapse(lt_synth,getImagesLib.vizParamsFalse,'Fitted LandTrendr Composite Timelapse')


Map.centerObject(landTrendrCollection.first())
Map.turnOnInspector()
Map.view()

Adding layer: LandTrendr All Predictors Time Series
Adding layer: Fitted LandTrendr Composite Timelapse
Starting webmap
Using default refresh token for geeView: /home/jupyter/.config/earthengine/credentials
Local web server at: http://localhost:1234/geeView/ already serving.
cwd /home/jupyter/lcms-training
Workbench Proxy URL: https://53c21733d8125e22-dot-us-west3.notebooks.googleusercontent.com/proxy/1234/geeView/?accessToken=None


127.0.0.1 - - [16/Aug/2023 17:37:21] "GET /geeView/?accessToken=None HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2023 17:37:21] "GET /geeView/js/runGeeViz.js HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2023 17:37:21] "GET /geeView/js/lcms-viewer.min.js HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2023 17:37:21] "GET /geeView/js/gena-gee-palettes.js HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2023 17:37:21] "GET /geeView/js/load.min.js HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2023 17:37:21] "GET /geeView/css/style.min.css HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2023 17:37:21] "GET /geeView/images/menu-hamburger_ffffff.svg HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2023 17:37:21] "GET /geeView/images/layer_icon.png HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2023 17:37:21] "GET /geeView/images/GEE.png HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2023 17:37:21] "GET /geeView/images/usfslogo.png HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2023 17:37:21] "GET /geeView/images/GEE_logo_transparent.png HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2023 1

In [16]:
# And then bring in CCDC data for each year
Map.clearMap()

ccdcTiles = ee.ImageCollection(export_ccdc_collection)
composites = ee.ImageCollection(export_composite_collection)
ccdcInfo = ccdcTiles.first().toDictionary().getInfo()
print(ccdcInfo)
startYear = 1984
endYear = 2023
startJulian = 1
endJulian = 365

ccdcImg = ccdcTiles.mosaic()
#Specify which harmonics to use when predicting the CCDC model
#CCDC exports the first 3 harmonics (1 cycle/yr, 2 cycles/yr, and 3 cycles/yr)
#If you only want to see yearly patterns, specify [1]
#If you would like a tighter fit in the predicted value, include the second or third harmonic as well [1,2,3]
whichHarmonics = [1,2,3]

#Whether to fill gaps between segments' end year and the subsequent start year to the break date
fillGaps = False

# Proportion of a year segment can be extrapolated
# This can be important if the CCDC run had a break toward the end of the available raw data, but has not fit a 
# new model. This would introduce a null value past that break date if set to 0
# By setting to e.g. 0.3, a segment at the end can be extended by 0.3 of a year before it will result in a null value
tEndExtrapolationPeriod = 0.3


# If annualizeWithCompositeDates = False, this month and day will be used as the time to pull CCDC values
yearStartMonth = 4
yearStartDay = 1


# When annualizing CCDC outputs, we can pull the CCDC fitted, harmonic model coefficients, 
# and fitted difference from the previous year using a set date (e.g. April 1) or we can use the date from within our annual
# composites to increase the chance the CCDC output matches the timing of the composite values used within
# LandTrendr. In this example, we can bring in the annual composites and pull the dates for annualizing CCDC 
# by setting annualizeWithCompositeDates = True
# This will slow down computing, so you can set it to False for testing runs
annualizeWithCompositeDates = True
compositeCollection = composites

annualSegCoeffs = changeDetectionLib.annualizeCCDC(ccdcImg, startYear, endYear, startJulian, endJulian, tEndExtrapolationPeriod, 
  yearStartMonth, yearStartDay, annualizeWithCompositeDates, compositeCollection)
annualCCDC = changeDetectionLib.getFitSlopeCCDC(annualSegCoeffs, startYear, endYear)
ccdc_bandNames = annualCCDC.first().bandNames()
ccdc_bandNames = ccdc_bandNames.removeAll(['year'])
print('Available CCDC Predictor Variables:',ccdc_bandNames.getInfo())

# Visualize output as you would a composite
Map.addLayer(annualCCDC.select(ccdc_bandNames),{'bands':'swir1_CCDC_fitted,nir_CCDC_fitted,red_CCDC_fitted','min':0.05,'max':0.6},'Annual CCDC Output')
Map.addLayer(annualCCDC.select(['year']),{},'CCDC Annualizing Time Input',False)
Map.turnOnInspector()
Map.view()


{'TileID': '90,54', 'TileSize': 60000}
Available CCDC Predictor Variables: ['blue_CCDC_INTP', 'blue_CCDC_SLP', 'blue_CCDC_COS1', 'blue_CCDC_SIN1', 'blue_CCDC_COS2', 'blue_CCDC_SIN2', 'blue_CCDC_COS3', 'blue_CCDC_SIN3', 'green_CCDC_INTP', 'green_CCDC_SLP', 'green_CCDC_COS1', 'green_CCDC_SIN1', 'green_CCDC_COS2', 'green_CCDC_SIN2', 'green_CCDC_COS3', 'green_CCDC_SIN3', 'red_CCDC_INTP', 'red_CCDC_SLP', 'red_CCDC_COS1', 'red_CCDC_SIN1', 'red_CCDC_COS2', 'red_CCDC_SIN2', 'red_CCDC_COS3', 'red_CCDC_SIN3', 'nir_CCDC_INTP', 'nir_CCDC_SLP', 'nir_CCDC_COS1', 'nir_CCDC_SIN1', 'nir_CCDC_COS2', 'nir_CCDC_SIN2', 'nir_CCDC_COS3', 'nir_CCDC_SIN3', 'swir1_CCDC_INTP', 'swir1_CCDC_SLP', 'swir1_CCDC_COS1', 'swir1_CCDC_SIN1', 'swir1_CCDC_COS2', 'swir1_CCDC_SIN2', 'swir1_CCDC_COS3', 'swir1_CCDC_SIN3', 'swir2_CCDC_INTP', 'swir2_CCDC_SLP', 'swir2_CCDC_COS1', 'swir2_CCDC_SIN1', 'swir2_CCDC_COS2', 'swir2_CCDC_SIN2', 'swir2_CCDC_COS3', 'swir2_CCDC_SIN3', 'NDVI_CCDC_INTP', 'NDVI_CCDC_SLP', 'NDVI_CCDC_COS1', 'NDVI

127.0.0.1 - - [16/Aug/2023 17:53:17] "GET /geeView/?accessToken=None HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2023 17:53:18] "GET /geeView/css/style.min.css HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2023 17:53:18] "GET /geeView/js/load.min.js HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2023 17:53:18] "GET /geeView/js/gena-gee-palettes.js HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2023 17:53:18] "GET /geeView/js/runGeeViz.js HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2023 17:53:18] "GET /geeView/js/lcms-viewer.min.js HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2023 17:53:18] "GET /geeView/images/GEE_logo_transparent.png HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2023 17:53:18] "GET /geeView/images/usfslogo.png HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2023 17:53:18] "GET /geeView/images/layer_icon.png HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2023 17:53:18] "GET /geeView/images/menu-hamburger_ffffff.svg HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2023 17:53:18] "GET /geeView/images/logos_usda-fs.svg HTTP/1.1" 200 -
127.0.0.1 - - [16/

In [20]:
# LCMS also uses various elevation/terrain variables to help our models
Map.clearMap()
terrainStack = []

# Can use any elevation data
# Will use SRTM for this example
# Resampling must be set to bicubic or bilinear or there will be artifacts in terrain derivatives
# Can use 'USGS/SRTMGL1_003' image for global applications
elevation = ee.Image('USGS/3DEP/10m').resample('bicubic')
slope = ee.Terrain.slope(elevation)
aspect = ee.Terrain.aspect(elevation)
sinAspect = aspect.sin().rename('sinAspect')
cosAspect = aspect.cos().rename('cosAspect')

terrainStack.extend([elevation,slope,sinAspect,cosAspect])

# A very simple algorithm for elevation position
# Good at finding ridges and depressions
def tpi(elevation,n):
    return elevation.subtract(elevation.focalMean(n)).rename([f'TPI_{int(n*2)}'])

# A nice palette from: https://github.com/gee-community/ee-palettes
# Use cmocean.speed[7]
palette = ['fffdcd', 'e1cd73', 'aaac20', '5f920c', '187328', '144b2a', '172313']

Map.addLayer(elevation.reproject(crs,transform),{'min':0,'max':250,'palette':palette},'Elevation')

# Get TPI for 11 and 21 pixel diameter circular kernels
for r in [5.5,10.5,20.5]:
    tpiR = tpi(elevation,r)
    terrainStack.append(tpiR)
    Map.addLayer(tpiR.reproject(crs,transform),{'min':-10,'max':10,'palette':palette},f'TPI {int(r*2)}')

# Stack all terrain data
terrainStack = ee.Image.cat(terrainStack)
print('Available terrain bands:',terrainStack.bandNames().getInfo())
Map.centerObject(timeSyncData.limit(2))
Map.turnOnInspector()
Map.view()

Adding layer: Elevation
Adding layer: TPI 11
Adding layer: TPI 21
Adding layer: TPI 41
Available terrain bands: ['elevation', 'slope', 'sinAspect', 'cosAspect', 'TPI_11', 'TPI_21', 'TPI_41']
Starting webmap
Using default refresh token for geeView: /home/jupyter/.config/earthengine/credentials
Local web server at: http://localhost:1234/geeView/ already serving.
cwd /home/jupyter/lcms-training
Workbench Proxy URL: https://53c21733d8125e22-dot-us-west3.notebooks.googleusercontent.com/proxy/1234/geeView/?accessToken=None


127.0.0.1 - - [16/Aug/2023 17:56:12] "GET /geeView/js/runGeeViz.js HTTP/1.1" 200 -


In [21]:
# Now extract LandTrendr, CCDC, and terrain values for each TimeSync plot from its respective year

# First, set up a function that will get all predictors for a given year
def getPredictorStack(yr):
     # Filter the LandTrendr and CCDC data for the given year
    lt_yr = lt_fit.filter(ee.Filter.calendarRange(yr,yr,'year')).first()
    ccdc_yr = annualCCDC.filter(ee.Filter.calendarRange(yr,yr,'year')).first()

    # Get a stack of all predictor bands
    extractionStack = ee.Image.cat([lt_yr,ccdc_yr,terrainStack])
    return extractionStack.set('system:time_start',ee.Date.fromYMD(yr,6,1).millis())

# Find the currently running tasks so we can check to see if something is already running or ready to run
currently_running_tasks =  tml.getTasks()

# Whether to overwrite existing tables
overwrite = False

# Extract for each year
for training_year in training_years:
    
    # Filter TimeSync data for the given year
    timeSyncData_yr = timeSyncData.filter(ee.Filter.eq('YEAR',training_year))
   
    # Get the predictor variables for the year
    extractionStack = getPredictorStack(training_year)
    
    # Extract the values
    extracted_values = extractionStack.reduceRegions(collection=timeSyncData_yr, reducer = ee.Reducer.first(), scale = None, crs = crs, crsTransform = transform, tileScale = 4)
    
    # Export them as an asset
    assetName = f'LCMS_Training_TimeSync_yr{training_year}_LandTrendr_CCDC_Terrain_Extraction'
    assetPath = f'{export_timeSync_folder}/{assetName}'
    
    currently_exporting = assetName in tml.getTasks()['running'] or assetName in tml.getTasks()['ready'] 
    currently_exists = aml.ee_asset_exists(assetPath)

    if overwrite and currently_exists:
        ee.data.deleteAsset(assetPath)
    if overwrite and currently_exporting:
        tml.cancelByName(assetName)
        
    # Export if it doesn't exist or isn't being run already
    if overwrite or (not currently_exists and not currently_exporting):
        t = ee.batch.Export.table.toAsset(collection = extracted_values,
                          description = assetName,
                          assetId = assetPath)
        print('Exporting:',assetName)
        print(t)
        t.start()
    else:
        print(f'{assetName} already exists or is currently being exported')

print('Done')

Exporting: LCMS_Training_TimeSync_yr2000_LandTrendr_CCDC_Terrain_Extraction
<Task EXPORT_FEATURES: LCMS_Training_TimeSync_yr2000_LandTrendr_CCDC_Terrain_Extraction (UNSUBMITTED)>
Exporting: LCMS_Training_TimeSync_yr2001_LandTrendr_CCDC_Terrain_Extraction
<Task EXPORT_FEATURES: LCMS_Training_TimeSync_yr2001_LandTrendr_CCDC_Terrain_Extraction (UNSUBMITTED)>
Exporting: LCMS_Training_TimeSync_yr2002_LandTrendr_CCDC_Terrain_Extraction
<Task EXPORT_FEATURES: LCMS_Training_TimeSync_yr2002_LandTrendr_CCDC_Terrain_Extraction (UNSUBMITTED)>
Exporting: LCMS_Training_TimeSync_yr2003_LandTrendr_CCDC_Terrain_Extraction
<Task EXPORT_FEATURES: LCMS_Training_TimeSync_yr2003_LandTrendr_CCDC_Terrain_Extraction (UNSUBMITTED)>
Exporting: LCMS_Training_TimeSync_yr2004_LandTrendr_CCDC_Terrain_Extraction
<Task EXPORT_FEATURES: LCMS_Training_TimeSync_yr2004_LandTrendr_CCDC_Terrain_Extraction (UNSUBMITTED)>
Exporting: LCMS_Training_TimeSync_yr2005_LandTrendr_CCDC_Terrain_Extraction
<Task EXPORT_FEATURES: LCMS_T

In [26]:
# Can track tasks here or at https://code.earthengine.google.com/tasks
# If you'd like to track the tasks, use this:
# tml.trackTasks2()

# If you want to cancel all running tasks, you can use this function
# tml.batchCancel()

# If you want to empty the collection of all images
# aml.batchDelete(export_timeSync_folder, type = 'tables')

print('done')


done
